In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.preprocessing.text import Tokenizer
import warnings
warnings.filterwarnings('ignore')

In [0]:
# change the path to read
path = '/content/drive/My Drive'
os.chdir(path)
os.listdir(path)

In [0]:
# Image Preparation

In [0]:
# extract features 
def extract_features(dataset):
    # load the model
    model = ResNet50() # change to ResNet 50
    # re-structure the model 
    model.layers.pop()  # delete the last/predict layer
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
                      # initialize a functional model (get the output of the middle layer)
    # summarize (output the parameters of each layer of the model)
    print(model.summary()) 
    # extract features
    features = dict() 
    for name in os.listdir(dataset): # return a list of files under the specified path
        # load an image from file
        filename = dataset + '/' + name
        image = load_img(filename, target_size=(224, 224, 3))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model 
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

        # prepare the image for the pre-trained model
        image = preprocess_input(image) # normalization (per-example mean subtraction) 
                                        # to improve the operation of the algorithm

        # get features
        feature = model.predict(image, verbose=0).flatten()
                            # verbose
        # get image id
        image_id = name.split('.')[0]
        # return features
        features[image_id] = feature
        # print('>%s' % name)
    return features

# extract features from all images
dataset = 'Images' # change 
features = extract_features(dataset)
print('Extracted Features: %d' % len(features)) # output: 8091

# save to file
from pickle import dump
dump(features, open('features2.pkl', 'wb'))

In [0]:
# Text Preparation

In [0]:
# load text into memory
filename = 'Flickr8k_text/Flickr8k.token.txt' 
# open the file as read only
file = open(filename,'r')
# read all text
text = file.read()
# close the file
file.close()

In [0]:
## Extract descriptions 

In [0]:
datatxt = []
for line in text.split('\n'):  
    line0 = line.split('\t')    
    if len(line0) == 1:
        continue
    w = line0[0].split('#')  
    datatxt.append(w + [line0[1].lower()])
# change to not capital writing
# datatxt contain n list0
df_txt = pd.DataFrame(datatxt,columns=["filename","index","desc"])

In [0]:
## Clean descriptions

In [0]:
# define the text_clean functions
import string
def remove_punctuation(text_original):
    text_no_punctuation = text_original.translate(str.maketrans('','',string.punctuation))
    return(text_no_punctuation)

def remove_single_character(text):
    text_len_more_than1 = ''
    for word in text.split(): # split word by white space/tokenize
        if len(word) > 1:
            text_len_more_than1 += ' ' + word
    return(text_len_more_than1)

def remove_numeric(text): 
    text_no_numeric = ''
    for word in text.split(): 
        isalpha = word.isalpha()
        if isalpha is False:
            print('{:10}: {:}'.format(word,isalpha)) # e.g. output: 111    : False
        if isalpha:  # if isalpha is True
            text_no_numeric += ' ' + word
    return(text_no_numeric)

In [0]:
# clean descriptions
def text_clean(text_original):
    text = remove_punctuation(text_original)
    text = remove_single_character(text)
    text = remove_numeric(text)
    return(text)


for i, desc in enumerate(df_txt.desc.values): 
    desc_new = text_clean(desc)
    df_txt['desc'].iloc[i] = desc_new

In [0]:
## Add start and end sequence tokens (prepare for LSTM)

In [0]:
from copy import copy
def add_start_end_seq_token(text):
    desc = []
    for txt in text:
        txt = 'startseq ' + txt + ' endseq'
        desc.append(txt)
    return(desc)
df_txt0 = copy(df_txt)
df_txt0['desc'] = add_start_end_seq_token(df_txt['desc'])
# preview
df_txt0.head(5)
# del df_txt

In [0]:
# load CNN weights
from pickle import load
images = load(open('features2.pkl', 'rb'))

In [0]:
# link image and text
df_txt6 = df_txt0
dimages, keepindex = [],[]

#print(df_txt6[:5])
for i in range(5):
    locals()['df_txt'+str(i)] = df_txt6.loc[df_txt6['index'].values == str(i),: ]
#locals()['df_txt'+str(0)] = df_txt6.loc[df_txt6['index'].values == '0',: ] 

# preview
#print(df_txt0['index'][:5])                                                                                       
#print(dtexts[:5])
print(df_txt0)

In [0]:
# link image and all the texts
def keep(df_txt1):                                                          
    for i, name in enumerate(df_txt1.filename):
        name = name.split('.')[0]
        if name in images.keys():
            dimages.append(images[name])
            keepindex.append(i)
    
    return dimages,keepindex

for i in range(5):
    locals()['dimages'+str(i)],locals()['keepindex'+str(i)] = keep(locals()['df_txt'+str(i)])
    locals()['fnames'+str(i)] = locals()['df_txt'+str(i)]['filename'].iloc[locals()['keepindex'+str(i)]].values
    locals()['dtexts'+str(i)] = locals()['df_txt'+str(i)]['desc'].iloc[locals()['keepindex'+str(i)]].values
    locals()['dimages'+str(i)] = np.array(locals()['dimages'+str(i)])

In [0]:
# tokenizer
from keras.preprocessing.text import Tokenizer

def token(dtexts1):
  num_words = 8000
  tokenizer = Tokenizer(num_words = 8000)
  tokenizer.fit_on_texts(dtexts1)  
  vocab_size1 = len(tokenizer.word_index) + 1
  print('vocabulary size : {}'.format(vocab_size1))
  dtexts1 = tokenizer.texts_to_sequences(dtexts1)
  print(dtexts1[:5])
  return dtexts1, vocab_size1
for i in range(5):
    locals()['dtexts'+str(i)],locals()['vocab_size'+str(i)] = token(locals()['dtexts'+str(i)])


In [0]:
# data partition
ptest, pval = 0.2, 0.2 

def split_test_val(dtexts1):# change the function name here
  N = len(dtexts1)
  Ntest, Nval = int(N*ptest), int(N*pval) 
  return(dtexts1[:Ntest], 
          dtexts1[Ntest:Ntest+Nval],  
          dtexts1[Ntest+Nval:])

for i in range(5):
    locals()['dt_test'+str(i)],locals()['dt_val'+str(i)],locals()['dt_train'+str(i)] = split_test_val(locals()['dtexts'+str(i)])
    locals()['di_test'+str(i)],locals()['di_val'+str(i)],locals()['di_train'+str(i)] = split_test_val(locals()['dimages'+str(i)])
    locals()['fnm_test'+str(i)],locals()['fnm_val'+str(i)],locals()['fnm_train'+str(i)] = split_test_val(locals()['fnames'+str(i)])
    locals()['maxlen'+str(i)] = np.max([len(text) for text in locals()['dtexts'+str(i)]])

    print('maxlen',i,locals()['maxlen'+str(i)])
    print('vocab_size',i,locals()['vocab_size'+str(i)])

maxlenchoose = maxlen1 # get the biggest maxlen
vocabchoose = vocab_size0 # get the biggest vocab size
#maxlen = np.max([len(text) for text in dtexts])

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def preprocessing(dtexts1,dimages): # change the function on maxlen, vocab_size
    N = len(dtexts1)
    print('descriptions = {}'.format(N))
    print('images = {}'.format(N))

    assert(N==len(dimages))
    Xtext, Ximage, ytext = [],[],[]
    for text,image in zip(dtexts1,dimages):
        for i in range(1,len(text)):
            in_text, out_text = text[:i], text[i]
            in_text = pad_sequences([in_text],maxlen = maxlenchoose).flatten() # flatten层: 扁平参数，防止过拟合
            out_text = to_categorical(out_text,num_classes = vocabchoose)
            Xtext.append(in_text)
            Ximage.append(image)
            ytext.append(out_text)
    Xtext  = np.array(Xtext)
    Ximage = np.array(Ximage)
    ytext  = np.array(ytext)
    print('{} {} {}'.format(Xtext.shape,Ximage.shape,ytext.shape))
    return(Xtext,Ximage,ytext)

# preprocessing is not necessary for testing data

# for loop way
for i in range(3): # RAM get boom when range to 5.Only 0,1,2 could be used at the same time
    print('training')
    locals()['Xtext_train'+str(i)],locals()['Ximage_train'+str(i)],locals()['ytext_train'+str(i)] = preprocessing(locals()['dt_train'+str(i)],locals()['di_train'+str(i)])
    print('\ntesting')
    locals()['Xtext_val'+str(i)],locals()['Ximage_val'+str(i)],locals()['ytext_val'+str(i)] = preprocessing(locals()['dt_val'+str(i)],locals()['di_val'+str(i)])

In [0]:
# Define model

In [0]:
# clean the sessions
import keras
keras.backend.clear_session()

In [0]:
from keras import layers

# image feature extractor model
input_image = layers.Input(shape=(Ximage_train1.shape[1],)) #4096
fimage1 = layers.Dropout(0.5)(input_image)
fimage2 = layers.Dense(256,activation='relu',name="ImageFeature")(fimage1)
# sequence model
input_txt = layers.Input(shape=(maxlenchoose,))
# embedding layer must be the first layer of the model
stxt1 = layers.Embedding(vocabchoose,256,mask_zero=True)(input_txt)
stxt2 = layers.Dropout(0.5)(stxt1)
stxt3 = layers.LSTM(256,name="TextFeature")(stxt2)
# decoder model
decoder1 = layers.add([fimage2,stxt3])
decoder2 = layers.Dense(256,activation='relu')(decoder1)
decoder3 = layers.Dense(256, activation='relu')(decoder2)
outputs = layers.Dense(vocabchoose,activation='softmax')(decoder3)
# combine the model 
model = Model(inputs=[input_image, input_txt],outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam') 
# summarize
print(model.summary())

In [0]:
# Train the model

In [0]:
import time
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
# fit model

start = time.time()

filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5' # the weight would be save to this file
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit([Ximage_train2, Xtext_train2], ytext_train2, 
                  epochs=2, verbose=2, 
                  batch_size=64, callbacks=[checkpoint],
                  validation_data=([Ximage_val2, Xtext_val2], ytext_val2))
      
end = time.time()
print("TIME TOOK {:3.2f}MIN".format((end - start )/60))

print(Ximage_train2.shape,Xtext_train2.shape,ytext_train2.shape)
# print(hist.history)

In [0]:
# visualize the loss
for label in ["loss","val_loss"]:
    plt.plot(hist.history[label],label=label)
plt.legend()
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [0]:
# load weights
model.load_weights('model-ep002-loss4.075-val_loss3.888.h5') # change to the best weight
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
scores=model.evaluate([Ximage_val2, Xtext_val2],ytext_val2,verbose=0)
print(model.metrics_names[1],scores[1])

In [0]:
# Prediction

In [0]:
# map an integer to a word
index_word = dict([(index,word) for word, index in tokenizer.word_index.items()])

In [0]:
# alternative
def index_word(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [0]:
# generate the descriptions
def predict_desc(image):
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(maxlen):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence],maxlen)
        # predict next word
        yhat = model.predict([image,sequence],verbose=0)
        # return the index(integer) of max(yhat)
        yhat = np.argmax(yhat)
        # map integer to word
        word = index_word[yhat] # alternative: word = index_word(yhat, tokenizer)
        # stop if cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if predict the end of the sequence
        if word == 'endseq':
            break
    return(in_text)

In [0]:
# demo: show five examples
npic = 5
npix = 224
target_size = (npix,npix,3)

count = 1
fig = plt.figure(figsize=(10,20))
for jpgfnm, image_feature in zip(fnm_test[:npic],di_test[:npic]):
    # images 
    filename = dataset + '/' + jpgfnm
    image_load = load_img(filename, target_size=target_size)
    ax = fig.add_subplot(npic,2,count,xticks=[],yticks=[])
    ax.imshow(image_load)
    count += 1

    # descriptions
    desc = predict_desc(image_feature.reshape(1,len(image_feature)))
    ax = fig.add_subplot(npic,2,count)
    plt.axis('off')
    ax.plot()
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.text(0,0.5,desc,fontsize=20)
    count += 1

plt.show()

In [0]:
# Evaluate the Model

In [0]:
from nltk.translate.bleu_score import sentence_bleu
# use BLEU for evaluation
nkeep = 5
pred_good, pred_bad, bleus = [], [], [] 
count = 0 
for jpgfnm, image_feature, tokenized_text in zip(fnm_test,di_test,dt_test):
    count += 1
    if count % 200 == 0:
        print("  {:4.2f}% is done..".format(100*count/float(len(fnm_test))))
    
    desc_true = [index_word[i] for i in tokenized_text]     
    desc_true = desc_true[1:-1] # remove startseq and endseq
  
    desc = predict_desc(image_feature.reshape(1,len(image_feature)))
    desc = desc.split()
    desc = desc[1:-1] # remove startseq and endseq
    
    bleu = sentence_bleu([desc_true],desc)
    bleus.append(bleu)
    if bleu > 0.7 and len(pred_good) < nkeep:
        pred_good.append((bleu,jpgfnm,desc_true,desc))
    elif bleu < 0.3 and len(pred_bad) < nkeep:
        pred_bad.append((bleu,jpgfnm,desc_true,desc))

In [0]:
print('The average accuracy based on BLEU is:',np.mean(bleus))

In [0]:
# demo: show the 'good' and 'bad' results
def plot_images(pred):
    def create_str(desc_true):
        line = ""
        for s in desc_true:
            line += " " + s
        return(line)
    
    npix = 224
    target_size = (npix,npix,3)    
    count = 1
    fig = plt.figure(figsize=(10,20))
    npic = len(pred)
    
    for pb in pred:
        bleu, jpgfnm, desc_true, desc = pb

        filename = dataset + '/' + jpgfnm
        image_load = load_img(filename, target_size=target_size) # load_img: 读取指定文件夹下所有images
        ax = fig.add_subplot(npic,2,count,xticks=[],yticks=[])
        ax.imshow(image_load)
        count += 1
        
        desc_true = create_str(desc_true)
        desc = create_str(desc)
        
        ax = fig.add_subplot(npic,2,count)
        plt.axis('off')
        ax.plot()
        ax.set_xlim(0,1)
        ax.set_ylim(0,1)
        ax.text(-0.1,0.5,"Prediction:" + desc,fontsize=20)
        ax.text(-0.1,0.75,"Original Description:" + desc_true,fontsize=20)
        ax.text(-0.1,0.25,"Accuray (BLEU): {}".format(bleu),fontsize=20)
        count += 1
    plt.show()

print("Bad Prediction")
plot_images(pred_bad)
print("Good Prediction")
plot_images(pred_good)